# Exploring Astronaut Activities in SQL

Welcome to your webinar workspace! You can follow along as we analyze the data in a SQL database and visualize the results.

To set up your integration, create a PostgreSQL integration with the following credentials:

- **Integration Name**: Astronaut Codealong
- **Hostname**: workspacedemodb.datacamp.com
- **Database**: astronauts
- **Username**: astronauts
- **Password**: astronauts

[Source of the data](https://www.kaggle.com/datasets/alexandrepetit881234/astronauts-extra-vehicular-activities)

_Note: You will need to set up your own integration to run the cells in this workspace!_

## Exploring our data
Let's start by looking at the table we will be working with.

In [10]:
SELECT
	*
FROM evas

,date,country,vehicle,duration,crew,purpose,year,program
0,1965-06-03,USA,Gemini IV,36,Ed White,First U.S. EVA. Used HHMU and took photos. G...,1965,Gemini
1,1966-03-16,USA,Gemini VIII,0,David Scott,HHMU EVA cancelled before starting by stuck on...,1966,Gemini
2,1966-06-05,USA,Gemini IX-A,127,Eugene Cernan,"Inadequate restraints, stiff 25ft umbilical an...",1966,Gemini
3,1966-07-19,USA,Gemini X,50,Mike Collins,Standup EVA. UV photos of stars. Ended by ey...,1966,Gemini
4,1966-07-20,USA,Gemini X,39,Mike Collins,Retrieved MMOD experiment from docked Agena. ...,1966,Gemini
...,...,...,...,...,...,...,...,...
370,2012-08-20,Russia,ISS Incr-32,351,"Gennady Padalka, Yuri Malenchenko",1 hr late start due to airlock valve. Relocat...,2012,ISS
371,2013-04-19,Russia,ISS Incr-35,398,"Pavel Vinogradov, Roman Romanenko","Installed plasma experiment/cables/probes, rep...",2013,ISS
372,2013-06-24,Russia,ISS Incr-36,394,"Fyodor Yurchikhin, Alexander Misurkin",Power cable clamps installed and Kurs tested i...,2013,ISS
373,2013-08-16,Russia,ISS Incr-36,449,"Fyodor Yurchikhin, Alexander Misurkin","Â Installed VINOSLIVOST experiment on MRM2, 2...",2013,ISS


Let's inspect the `purpose` column in greater detail.

In [11]:
SELECT
	purpose
FROM evas

,purpose
0,First U.S. EVA. Used HHMU and took photos. G...
1,HHMU EVA cancelled before starting by stuck on...
2,"Inadequate restraints, stiff 25ft umbilical an..."
3,Standup EVA. UV photos of stars. Ended by ey...
4,Retrieved MMOD experiment from docked Agena. ...
...,...
370,1 hr late start due to airlock valve. Relocat...
371,"Installed plasma experiment/cables/probes, rep..."
372,Power cable clamps installed and Kurs tested i...
373,"Â Installed VINOSLIVOST experiment on MRM2, 2..."


## What are the most common types of EVAs?
Using [`CASE` expressions](https://www.postgresql.org/docs/current/functions-conditional.html), let's start to get a rough idea of the most popular types of EVAs.

In [18]:
 SELECT
    purpose,
    CASE WHEN purpose ILIKE '%photo%' THEN 1 ELSE 0 END AS photography,
    CASE WHEN purpose ILIKE '%collect%' THEN 1 ELSE 0 END AS collection,        
    CASE WHEN purpose ILIKE '%construct%' OR purpose ILIKE '%install%' OR purpose ILIKE '%assembl%' THEN 1 ELSE 0 END AS installation,
    CASE WHEN purpose ILIKE '%recover%' OR purpose ILIKE '%retrieve%' THEN 1 ELSE 0 END AS retrieval,
    CASE WHEN purpose ILIKE '%jettison%' THEN 1 ELSE 0 END AS jettison,
    CASE WHEN purpose ILIKE '%replace%' OR purpose ILIKE '%fix%' OR purpose ILIKE '%repair%' OR purpose ILIKE '%servic%' THEN 1 ELSE 0 END AS repair
FROM evas

,purpose,photography,collection,installation,retrieval,jettison,repair
0,First U.S. EVA. Used HHMU and took photos. G...,1,0,0,0,1,0
1,HHMU EVA cancelled before starting by stuck on...,0,0,0,0,0,0
2,"Inadequate restraints, stiff 25ft umbilical an...",0,0,0,0,0,0
3,Standup EVA. UV photos of stars. Ended by ey...,1,0,0,0,0,0
4,Retrieved MMOD experiment from docked Agena. ...,0,0,0,1,0,0
...,...,...,...,...,...,...,...
370,1 hr late start due to airlock valve. Relocat...,0,0,1,1,0,1
371,"Installed plasma experiment/cables/probes, rep...",0,0,1,1,1,1
372,Power cable clamps installed and Kurs tested i...,1,0,1,0,0,1
373,"Â Installed VINOSLIVOST experiment on MRM2, 2...",0,0,1,0,0,0


We are now ready to build this into a final query!

In [2]:
WITH purposes AS ( 
 	SELECT
        purpose,
        CASE WHEN purpose ILIKE '%photo%' THEN 1 ELSE 0 END AS photography,
        CASE WHEN purpose ILIKE '%collect%' THEN 1 ELSE 0 END AS collection,
        CASE WHEN purpose ILIKE '%construct%' OR purpose ILIKE '%install%' OR purpose ILIKE '%assembl%' THEN 1 ELSE 0 END AS installation,
        CASE WHEN purpose ILIKE '%recover%' OR purpose ILIKE '%retrieve%' THEN 1 ELSE 0 END AS retrieval,
        CASE WHEN purpose ILIKE '%jettison%' THEN 1 ELSE 0 END AS jettison,
        CASE WHEN purpose ILIKE '%replace%' OR purpose ILIKE '%fix%' OR purpose ILIKE '%repair%' OR purpose ILIKE '%servic%' THEN 1 ELSE 0 END AS repair
    FROM evas
)

SELECT 
	SUM(photography) AS count,
    'photography' AS type
FROM purposes
UNION
SELECT 
	SUM(collection) AS count,
    'collection' AS type
FROM purposes
UNION
SELECT 
	SUM(retrieval) AS count,
    'retrieval' AS type
FROM purposes
UNION
SELECT 
	SUM(jettison) AS count,
    'jettison' AS type
FROM purposes
UNION
SELECT 
	SUM(repair) AS count,
    'repair' AS type
FROM purposes
UNION
SELECT 
	SUM(installation) AS count,
    'installation' AS type
FROM purposes
ORDER BY count

,count,type
0,16,collection
1,37,jettison
2,43,photography
3,79,retrieval
4,129,repair
5,191,installation


In [11]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,count,type
0,16,collection
1,37,jettison
2,43,photography
3,79,retrieval
4,129,repair


## How much material has been extracted from EVAs?
We also saw numerous references to extracting rock/dust or geological material when skimming through the purpose column. In this case, extracting the total quantity across the columns will be difficult. Regular expressions to the rescue!

We will define a pattern to extract the total pounds collected per EVA, and then sum them up. Let's first do a sense check of the data.

In [94]:
SELECT
	purpose
FROM evas
WHERE purpose ILIKE '%geologic%' OR purpose ILIKE '%rock%'

,purpose
0,First to walk on the moon. Some trouble getti...
1,Collected 75.6 lb of geologic material. ALSEP...
2,Collected 94.4 lb of geologic material. ALSEP...
3,Collected 169 lb of geologic material. ALSEP ...
4,Collected 208 lb of rock/dust (41lb this day)....
5,Collected 82 lb of rock/dust. Drove rover 11.5 km
6,Collected 90 lb of rock/dust. Drove rover 27....
7,Collected 243 lb of geologic material. ALSEP ...


Okay, we now know that the format of the pounds extracted is always `"number lbs of rock/geologic"`. We can construct a pattern to detect this and extract the number!

To do so, we will make use of:
- `\d+` to match one or more digits.
- `\.?` to match zero or more periods.\
- `*` to match zero or more digits following the optional decimal place.
- `()` to specify we only want the digits.
- `\s` to match the whitespace (i.e., spaces).
- `|` to specify we either want to match "geologic" or "rock".

Let's put this into action, using [`SUBSTRING()`](https://www.postgresql.org/docs/9.1/functions-string.html) to extract our pattern!

In [16]:
SELECT
	purpose,
	SUBSTRING(purpose, '(\d+\.?\d*)\slb\sof\s((rock)|(geologic))') AS weight
FROM evas
WHERE purpose ILIKE '%rock%' OR purpose ILIKE '%geologic%'

,purpose,weight
0,First to walk on the moon. Some trouble getti...,46.3
1,Collected 75.6 lb of geologic material. ALSEP...,75.6
2,Collected 94.4 lb of geologic material. ALSEP...,94.4
3,Collected 169 lb of geologic material. ALSEP ...,169.0
4,Collected 208 lb of rock/dust (41lb this day)....,208.0
5,Collected 82 lb of rock/dust. Drove rover 11.5 km,82.0
6,Collected 90 lb of rock/dust. Drove rover 27....,90.0
7,Collected 243 lb of geologic material. ALSEP ...,243.0


In [17]:
WITH weights AS (
    SELECT
        purpose,
        SUBSTRING(purpose, '(\d+\.?\d+)\slb\sof\s[rock|geologic]')::NUMERIC AS weight
    FROM evas
    WHERE purpose ILIKE '%rock%' OR purpose ILIKE '%geologic%'
)

SELECT SUM(weight)
FROM weights

,sum
0,1008.3


## Which astronauts have the most time in EVAs?
We also have information on how much time each EVA took and the astronauts who participated. Let's use this information to try and calculate totals for each astronaut!

First, let's see the maximum number of astronauts in an EVA by [splitting](https://www.postgresql.org/docs/9.1/functions-string.html) the crew column. We can also use TRIM() to remove any extra whitespace from the column.

In [6]:
SELECT 
	SPLIT_PART(crew, ',', 3) AS third_astronaut
FROM evas
WHERE SPLIT_PART(crew, ',', 3) != ''

Now we know how we can extract the time for each astronaut! Let's create a CTE we can then use to piece together `duration` information for each astronaut.

In [10]:
WITH astronauts_split AS (
    SELECT
    	crew,
        SPLIT_PART(crew, ',', 1) AS first_astronaut,
        SPLIT_PART(crew, ',', 2) AS second_astronaut,
        SPLIT_PART(crew, ',', 3) AS third_astronaut,
        duration
    FROM evas
)

SELECT *
FROM astronauts_split

Now it's just a matter of splitting apart this table and merging the results.

In [2]:
WITH astronauts_split AS (
    SELECT
    	crew,
        SPLIT_PART(crew, ',', 1) AS first_astronaut,
        SPLIT_PART(crew, ',', 2) AS second_astronaut,
        SPLIT_PART(crew, ',', 3) AS third_astronaut,
        duration
    FROM evas
),

astronaut_durations AS (
    SELECT
        first_astronaut AS astronaut,
        duration
    FROM astronauts_split
    WHERE first_astronaut != ''
    UNION ALL 
    SELECT
        second_astronaut AS astronaut,
        duration
    FROM astronauts_split
    WHERE second_astronaut != ''
    UNION ALL 
    SELECT
        third_astronaut AS astronaut,
        duration
    FROM astronauts_split
    WHERE third_astronaut != ''
)

SELECT
	astronaut,
    SUM(duration) AS total_duration
FROM astronaut_durations
GROUP BY astronaut
ORDER BY total_duration DESC
LIMIT 10

,astronaut,total_duration
0,Jerry Ross,3501
1,Anatoly Solovyev,3086
2,Scott Parazynski,2825
3,Nikola Budarin,2672
4,John Grunsfeld,2527
5,Mike Lopez-Alegria,2501
6,Mike Fincke,2472
7,Dan Tani,2351
8,Victor Afanasyev,2314
9,Rick Mastracchio,2311


In [3]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,astronaut,total_duration
0,Jerry Ross,3501
1,Anatoly Solovyev,3086
2,Scott Parazynski,2825
3,Nikola Budarin,2672
4,John Grunsfeld,2527
5,Mike Lopez-Alegria,2501
6,Mike Fincke,2472
7,Dan Tani,2351
8,Victor Afanasyev,2314
9,Rick Mastracchio,2311


## What is the cumulative amount of time spent in EVAs over time?
Finally, let's take a look at the cumulative time spent in EVAs by year and space program. To do so, we will need to use a [window function](https://www.postgresql.org/docs/current/tutorial-window.html) in combination with a subquery.

In [1]:
SELECT
	TO_DATE(year::TEXT, 'YYYY') AS year,
    program,
    duration,
    SUM(duration) OVER(PARTITION BY program ORDER BY year) AS cumulative_duration
FROM (
	SELECT
        year,
        program,
        SUM(duration) AS duration
    FROM evas
    GROUP BY year, program
    ORDER BY year, program
) AS sub
ORDER BY year, program

,year,program,duration,cumulative_duration
0,1965-01-01 00:00:00+00:00,Gemini,36,36
1,1965-01-01 00:00:00+00:00,Voskhod,12,12
2,1966-01-01 00:00:00+00:00,Gemini,720,756
3,1969-01-01 00:00:00+00:00,Apollo,707,707
4,1969-01-01 00:00:00+00:00,Soyuz,37,37
...,...,...,...,...
63,2010-01-01 00:00:00+00:00,Space Shuttle,3591,61140
64,2011-01-01 00:00:00+00:00,ISS,1388,17901
65,2011-01-01 00:00:00+00:00,Space Shuttle,2492,63632
66,2012-01-01 00:00:00+00:00,ISS,2009,19910


In [2]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,year,program,duration,cumulative_duration
0,1965-01-01 00:00:00+00:00,Gemini,36,36
1,1965-01-01 00:00:00+00:00,Voskhod,12,12
2,1966-01-01 00:00:00+00:00,Gemini,720,756
3,1969-01-01 00:00:00+00:00,Apollo,707,707
4,1969-01-01 00:00:00+00:00,Soyuz,37,37
...,...,...,...,...
63,2010-01-01 00:00:00+00:00,Space Shuttle,3591,61140
64,2011-01-01 00:00:00+00:00,ISS,1388,17901
65,2011-01-01 00:00:00+00:00,Space Shuttle,2492,63632
66,2012-01-01 00:00:00+00:00,ISS,2009,19910
